In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Get The Data

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip -o /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip -o /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
data = pd.read_csv('train.tsv', sep = '\t')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

# Cleaning Data

In [ ]:
pd.isnull(data).sum()

In [ ]:
data.brand_name = data.brand_name.fillna(value = "No brand defined")

data.category_name = data.category_name.fillna(value = "Unknown/Unknown/Unknown")

data.item_description = data.item_description.fillna(value = "No description yet")

In [ ]:
pd.isnull(data).sum()

# Explore Data Analytical

In [ ]:
import numpy as np

# For illustration only. Sklearn has train_test_split()
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
def split(txt):
    try :
        return txt.split("/")
    except :
        return ("Unknown", "Unknown", "Unknown")

In [ ]:
data['general_category']='' 
data['subcategory_1'] = '' 
data['subcategory_2'] = ''

In [ ]:
data['general_category'],data['subcategory_1'],data['subcategory_2'] = \
zip(*data['category_name'].apply(lambda x: split(x)))

In [ ]:
data

In [ ]:
data.drop(columns =["category_name"], inplace = True)

data

In [ ]:
data['item_condition_id'].value_counts().plot(kind='bar')

In [ ]:
data['general_category'].value_counts().plot(kind='bar')

In [ ]:
data['subcategory_1'].value_counts().plot(kind='bar')

In [ ]:
data['subcategory_2'].value_counts().plot(kind='bar')

In [ ]:
log_prices = np.log1p(data.price)
log_prices.hist()

# Prepare Data

In [ ]:
cat_col = data.select_dtypes(include=['object']).columns.tolist()
cat_col

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X=LabelEncoder()
for col in cat_col:
    data[col] = labelencoder_X.fit_transform(data[col])
data[cat_col]

In [ ]:
corr_matrix = data.corr()
corr_matrix

In [ ]:
import seaborn as sns

sns.heatmap(corr_matrix, xticklabels=corr_matrix.columns.values, yticklabels=corr_matrix.columns.values)

In [ ]:
if 'price'in data.columns:
    data_targets = data.pop('price')
data_targets

# Spliting Data

In [ ]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target = train_test_split(data,data_targets, test_size=0.2)

In [ ]:
train_data

In [ ]:
train_data.shape

In [ ]:
train_target.shape

In [ ]:
test_data.shape

In [ ]:
test_target.shape

# Linear Models & sklearn

In [ ]:
from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,Ridge
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
def checkModelPerformane(model):
    X=train_data
    y=train_target
    model.fit(X,y)
    pred=model.predict(test_data)
    print("mean_squared_error: ",np.sqrt(mean_squared_error(test_target, pred))) 
    print("mean_absolute_error: ", np.sqrt(mean_absolute_error(test_target, pred)))

In [ ]:
print("LinearRegression")
checkModelPerformane(LinearRegression())

In [ ]:
print("LinearRegression with lasso regualrizion")
checkModelPerformane(linear_model.Lasso(alpha=0.1))

In [ ]:
print("lasso regression ")
checkModelPerformane(linear_model.Lasso(alpha=0.1))

In [ ]:
print("ElasticNet regression ")
checkModelPerformane(ElasticNet())

In [ ]:
print("Ridge regression ")
checkModelPerformane(Ridge(alpha=1.0))